## <font color=red> You should not import any new libraries. Your code should run with python=3.x</font>
## <font color=red> Please don't rename this .ipynb file.</font><br>
- Your solutions will be auto-graded. Hence we request you to follow the instructions.
- Modify the code only between 
```
## TODO
## END TODO
```
- In addition to above changes, you can play with arguments to the functions for generating plots
- We will run the auto grading scripts with private test data

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from scipy import linalg

## Please make sure that your code works with loading data from relative path only

i.e. ```pd.read_csv('./data/multi_var_lasso.csv')``` should not throw an error when we run the auto-grading scripts

In [ ]:
data_multi = pd.read_csv('./data/multi_var_lasso.csv')
cols = [f"x_gt_{idx}" for idx in range(1, 6)]
X_multi = np.array(data_multi[cols])
Y_multi = np.array(data_multi['y_gt'])

In [ ]:
data_multi_class = pd.read_csv('./data/3_class_perceptron.csv')
cols = [f"x_gt_{idx}" for idx in range(1, 3)]
X_multi_class = np.array(data_multi_class[cols])
Y_multi_class = np.array(data_multi_class['y_gt'])

In [ ]:
def mse_multi_var(X, Y, w, b):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 1)
    Y - numpy array of shape (n_samples, 1)
    w - a float
    b - a float
    '''

    ## TODO
    N = X.shape[0]
    mse = np.square(np.dot(X, w) + b - Y).mean() / N

    ## END TODO

    return mse


In [ ]:
def mse_regularized(X, Y, w, b, lamda):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 1)
    Y - numpy array of shape (n_samples, 1)
    w - a float
    b - a float
    '''
    ## TODO
    N = X.shape[0]
    mse = lamda * np.square(w).mean() + \
        np.square(np.dot(X, w) + b - Y).mean() / N

    ## END TODO

    return mse


## Plot Graphs

- This function plots the ground truth curve in <font color=green>green</font> and the predicted function in <font color=red>red</font>


In [ ]:
def plot_curves(w, b, x, y):
    x_gt = np.linspace(-1, 2, 50)
    y_gt = 1 - 3 * x_gt - 2 * x_gt ** 2 + 2.5 * x_gt ** 3
    # print(x_gt.shape,y_gt.shape)
    if len(w) == 1:
        y_fit = w * x_gt + b
    elif len(w) == 5:
        x_fit = x_gt
        for pow in range(2, 4):
            x_fit = np.vstack([x_fit, np.power(x_gt, pow)])

        x_fit = np.vstack([x_fit, np.power(x_gt, 2)])
        x_fit = np.vstack([x_fit, np.power(x_gt, 1)])

        y_fit = np.dot(w, x_fit) + b
    else:
        assert False, 'Pass a valid w'
    plt.plot(x_gt, y_gt, color="green", label='1 - 3 * x - 2 * x ** 2 + 2.5 * x ** 3')
    plt.plot(x_gt, y_fit, color='red', label="Fitted Function y = w.Tx + b")
    if len(x.shape) == 1:
        x_plot = np.vstack([x, np.ones(len(x))]).T
        # print (x_plot.shape, y.shape)
    else:
        x_plot = x
    plt.scatter(x_plot[:, 0], y)
    plt.legend()
    plt.title("Ground Truth Function")
    plt.show()


In [ ]:
def split_data(X, Y, train_ratio=0.6):
    '''
    Split data into train and validation sets such that train
    contains floor(n_sampes * train_ratio) and test contains the remaining
    samples

    Args:
    X - numpy array of shape (n_samples, n_features)
    Y - numpy array of shape (n_samples, 1)
    train_ratio - fraction of samples to be used as training data

    Returns:
    X_train, Y_train, X_val, Y_val
    '''
    ## TODO
    n_samples = X.shape[0]
    train_size = int(n_samples * train_ratio)

    np.random.seed(0)
    indices = np.random.permutation(n_samples)
    X_train = X[indices[:train_size]]
    Y_train = Y[indices[:train_size]]
    X_val = X[indices[train_size:]]
    Y_val = Y[indices[train_size:]]

    ## END TODO

    return X_train, Y_train, X_val, Y_val


# Lasso Regression


In [ ]:
X_train, Y_train, X_val, Y_val = split_data(X_multi, Y_multi, train_ratio=0.6)


def soft(x, threshold):
    ind_high = x > threshold
    ind_low = x < -threshold
    res = np.zeros_like(x)
    res[ind_high] = x[ind_high] - threshold
    res[ind_low] = x[ind_low] + threshold
    return res


def ista(X_train, Y_train, X_val, Y_val, epochs=100, lr=1e-3, lamda=1):
    '''
    Perform multi variable lasso regression using ISTA

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    epochs - number of gradient descent steps
    lr - learnig rate
    lamda - regularization_weight
    '''

    w = np.zeros(X_train.shape[1])  # np.random.randn(X_train.shape[1])
    b = 0
    ## TODO
    N = X_train.shape[0]
    for i in range(epochs):
        grad_w = 2 / N * np.dot(np.dot(X_train, w) + b - Y_train, X_train)
        grad_b = 2 / N * np.sum(np.dot(X_train, w) + b - Y_train)
        w_LS = w - lr * grad_w
        b_LS = b - lr * grad_b
        w = soft(w_LS, lr * lamda)
        b = soft(b_LS, lr * lamda)
        print(w, w_LS)

    ## END TODO

    mse_train = mse_regularized(X_train, Y_train, w, b, lamda)
    mse_val = mse_regularized(X_val, Y_val, w, b, lamda)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    print(w.shape, b, X_train.shape, Y_train.shape)
    plot_curves(list(w), b, X_train, Y_train)
    return w, b


ista(X_train, Y_train, X_val, Y_val)


# Ridge Regression


In [ ]:
def multivar_reg_closedform(X_train, Y_train, X_val, Y_val, lamda=0.5):
    '''
    Perform L2 regularized multi variable least squares regression using 
    closed form update rules

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    lambda - regularization weight
    '''

    w = np.zeros(X_train.shape[1])
    b = 0

    ## TODO
    # use closed-form solution from previous assignment
    N = X_train.shape[0]
    phi = np.hstack((np.ones((N, 1)), X_train))
    weight = lamda * N * np.eye(X_train.shape[1] + 1)
    weight[0, 0] = 0
    w_all = linalg.inv(phi.T @ phi + weight) @ phi.T @ Y_train
    w = w_all[1:]
    b = w_all[0]

    ## END TODO

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    plot_curves(list(w), b, X_train, Y_train)

    return w, b


multivar_reg_closedform(X_train, Y_train, X_val, Y_val)


# Bias-Variance Tradeoff


In [ ]:
#
def ridge(X_train, Y_train, X_test, epochs=20, lr=2e-1, lamda=1):

    w = 0.6
    b = 2
    n = float(len(X_train))  # Number of elements in X

    # Performing Gradient Descent
    ## TODO

    ## END TODO

    Y_pred = w*X_test+b
    return Y_pred


def lasso(X_train, Y_train, X_test, epochs=20, lr=2e-1, lamda=1):
    w = 0.6
    b = 2
    n = float(len(X_train))  # Number of elements in X

    # Performing Gradient Descent
    ## TODO

    ## END TODO

    Y_pred = w*X_test+b
    return Y_pred


def ols(X_train, Y_train, X_test, epochs=20, lr=2e-1):

    w = 0.6
    b = 2
    n = float(len(X_train))  # Number of elements in X

    # Performing Gradient Descent
    ## TODO

    ## END TODO

    Y_pred = w*X_test+b
    return Y_pred


In [ ]:
# We fit multiple lines onto our linear model defined by y = c + mx + error.
num_lines = 10
# size of Dataset, len(Dataset) for 1 line
n = 500
# weights (slope/intercept)
c = 3
m = 5


In [ ]:
def gen_data():
    '''
    We sample n, X from uniform distribution and error from zero mean normal distribtion, we find Y using mx+c+e
    We do the above for num_lines number of time, to fit different lines for each models(lasso, ols, ridge), doing so will
    be helpfull in calculating Expected value of learned esitmator of all n*num_lines dataset

    DO NOT CHANGE this function.
    '''

    w0 = c
    w1 = m
    data = np.zeros(shape=(num_lines, n, 2))
    for i in range(num_lines):
        x = np.random.uniform(-2, 2, size=(n, 1))
        e = np.random.normal(0, 8, size=(n, 1))
        y = w0 + w1 * x + e
        x_y = np.concatenate((x, y), axis=1)
        data[i, :, :] = x_y
    return np.array(data)


In [ ]:
def gen_bais_variance(Y_preds, Y_true):
    ## TODO

    ## END TODO
    return bias, variance


In [ ]:
def make_prediction(data, X_test, lambda_=0.5):

    y_hat = np.zeros(shape=(num_lines, 3))  # store prediction of all model

    for i in range(num_lines):
        X = data[i, :, 0].reshape(n, 1)
        y = data[i, :, 1].reshape(n, 1)

        y_hat[i, 0] = ols(X, y, X_test)

        y_hat[i, 1] = ridge(X, y, X_test, lamda=lambda_)

        y_hat[i, 2] = lasso(X, y, X_test, lamda=lambda_)

    return y_hat


In [ ]:
def plot_figure(l, b, v):

    plt.plot(l, b[:, 0], label="OLS")
    plt.plot(l, b[:, 1], label="Ridge")
    plt.plot(l, b[:, 2], label="Lasso")
    plt.xlabel("Regularization coefficient")
    plt.ylabel("Bias")
    plt.title("Bias vs Lambda")
    plt.legend(loc="upper left")
    plt.show()

    plt.plot(l, v[:, 0], label="OLS")
    plt.plot(l, v[:, 1], label="Ridge")
    plt.plot(l, v[:, 2], label="Lasso")
    plt.xlabel("Regularization coefficient")
    plt.ylabel("Variance")
    plt.title("Variance vs Lambda")
    plt.legend(loc="lower left")
    plt.show()


In [ ]:
def driver():
    all_lambda = np.linspace(0.0001, 1, 100)
    all_bias = np.zeros(shape=(len(all_lambda), 3))
    all_variance = np.zeros(shape=(len(all_lambda), 3))
    dataset = gen_data()
    X_test = np.array([4]).reshape((1, 1))
    for i, l in enumerate(all_lambda):
        Y_hat = make_prediction(dataset, X_test, lambda_=l)
        ## TODO
        # calculate true mean Y_true

        ## END TODO
        all_bias[i, :], all_variance[i, :] = gen_bais_variance(Y_hat, Y_true)
    plot_figure(all_lambda, all_bias, all_variance)


driver()


# Perceptron


In [ ]:
X_train_multiclass, Y_train_multiclass, X_val_multiclass, Y_val_multiclass = split_data(
    X_multi_class, Y_multi_class, train_ratio=0.6)


def perceptron_algo(X_train_multiclass, Y_train_multiclass, X_val_multiclass, Y_val_multiclass, n_class=3, epochs=30):
    W = np.zeros((n_class, len(X_train_multiclass[0])))
    ## TODO

    ## END TODO

    Y_pred = list()
    for i in range(len(Y_val_multiclass)):
        y_pred = np.argmax([np.dot(W[0], X_val_multiclass[i]), np.dot(
            W[1], X_val_multiclass[i]), np.dot(W[2], X_val_multiclass[i])])
        Y_pred.append(y_pred)

    from sklearn.metrics import classification_report
    target_names = ['class 0', 'class 1', 'class 2']
    print(classification_report(Y_val_multiclass, Y_pred, target_names=target_names))
    return W


perceptron_algo(X_train_multiclass, Y_train_multiclass, X_val_multiclass, Y_val_multiclass)
